In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from transformers import TFBertModel,  BertConfig, BertTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
import pickle
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# Freeze the randomness
from numpy.random import seed
seed(1337)
import tensorflow as tf
tf.random.set_seed(42)

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
globalStrategy = "retrain" # Either retrain or keep

Init function to print f1 score and Jaccard score

In [5]:
def scoring(data, predict):
    f1 = []
    jaccard = []
    threshold = []
    for i in np.arange(0.30, 0.99, 0.01):
        predict_ = np.where(predict >= i, 1, 0)
        f1_micro = metrics.f1_score(data, predict_, average = 'micro')
        jaccard_micro = metrics.jaccard_score(data, predict_, average = 'micro')
        f1.append(f1_micro)
        jaccard.append(jaccard_micro)
        threshold.append(i)
    
    results = pd.DataFrame()
    results['Threshold'] = threshold
    results['F1_micro'] = f1
    results['Jaccard_micro'] = jaccard
    results = results[results['F1_micro'] == results['F1_micro'].max()]
    return results

Defining F1 Metric by hand as Keras does not support it naturally now

https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

In [6]:
data = pd.read_csv('data_cleaned.csv',
                   converters={"preprocessedTags": lambda x: x.strip("[]").replace("'","").split(", ")})
#data = pd.read_csv('data_cleaned2.csv',
#                   converters={"preprocessedTags": lambda x: x.strip("[]").replace("'","").split(", ")})

data = data[['desc', 'unstemmed_desc', 'preprocessedTags', 'Tag1']].dropna()
data.head()

,desc,unstemmed_desc,preprocessedTags,Tag1
0,vs cakephp vs zend vs cakephp vs zend cakephp ...,vs cakephp vs zend vs cakephp vs zend cakephp ...,[php],php
1,tool generat mock data tool generat mock data ...,tools generating mock data tools generating mo...,[testing],testing
2,laravel use statement non name cach effect lar...,laravel use statement non name cache effect la...,"[php, laravel]",php
3,add client authent add client authent server r...,add client authentication add client authentic...,[java],java
4,variabl error variabl error system namespac cl...,variable error variable error system namespace...,[c#],c#


Divide dataset into X and Y values, and transform Y to numeric

In [7]:
x = data['desc']
x_unstemmed = data['unstemmed_desc']
y = data['preprocessedTags']
# y_tag = data['Tag1']
mb = MultiLabelBinarizer()
y_encoded = mb.fit_transform(y)
# y_encoded = mb.fit_transform(y_tag)

# Baseline

## TF-IDF

In [8]:
max_features = 750
tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(1, 2))
tfidf.fit(x)
tfidf.transform(x)

tfidf_vector_X = tfidf.transform(x).toarray()

In [9]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_vector_X, 
                                                                            y_encoded, 
                                                                            test_size=0.2, 
                                                                            random_state=42)

In [10]:
x_train_tfidf.shape[1:]

(750,)

## Neural Network

In [11]:
# Define the model
xInput = Input(shape=(x_train_tfidf.shape[1:]))
x_ = Dense(700, activation='relu')(xInput)
x_ = Dense(700, activation='relu')(x_)
output = Dense(len(y_train_tfidf[1]), activation='sigmoid')(x_)

baseline = Model(inputs=xInput, outputs=output, name='Baseline')

# Compile the model
baseline.compile(loss=CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
                 optimizer=Adam(learning_rate=0.0000001),
                 metrics=['acc'])

In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10,
                                            mode='max',
                                            restore_best_weights=True)

In [13]:
baseline.summary()

Model: "Baseline"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 750)]             0         
_________________________________________________________________
dense (Dense)                (None, 700)               525700    
_________________________________________________________________
dense_1 (Dense)              (None, 700)               490700    
_________________________________________________________________
dense_2 (Dense)              (None, 206)               144406    
Total params: 1,160,806
Trainable params: 1,160,806
Non-trainable params: 0
_________________________________________________________________


In [14]:
epochs = 200
batch_size = 128

In [15]:
# Load the baseline, if does not exist then train one
if globalStrategy == 'retrain' or globalStrategy == 'retrainBaseline':
    epochs = epochs
    batch_size=batch_size
    history = baseline.fit(x_train_tfidf, y_train_tfidf,
                           epochs=epochs,
                           validation_split=0.1,
                           callbacks=[callback],
                           verbose=1)

    baseline.save('/home/mlmaster/Code/Ing_ml_P7/Baseline/')
    baseline = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/Baseline/')
else:
    try:
        baseline = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/Baseline/')
    except OSError:
        epochs = epochs
        batch_size=batch_size
        history = baseline.fit(x_train_tfidf, y_train_tfidf,
                               epochs=epochs,
                               validation_split=0.1,
                               callbacks=[callback],
                               verbose=1)

        baseline.save('/home/mlmaster/Code/Ing_ml_P7/Baseline/')
        baseline = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/Baseline/')

Epoch 1/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.6139 - acc: 0.0062 - val_loss: 9.7354 - val_acc: 0.0050
Epoch 2/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.5843 - acc: 0.0082 - val_loss: 9.7322 - val_acc: 0.0074
Epoch 3/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.5100 - acc: 0.0102 - val_loss: 9.7291 - val_acc: 0.0093
Epoch 4/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.5734 - acc: 0.0126 - val_loss: 9.7260 - val_acc: 0.0122
Epoch 5/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.5252 - acc: 0.0146 - val_loss: 9.7228 - val_acc: 0.0130
Epoch 6/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.4963 - acc: 0.0186 - val_loss: 9.7196 - val_acc: 0.0194
Epoch 7/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.5514 - acc: 0.0215 - val_loss: 9.7164 - val_acc: 0.0252
Epoch 8/200
1059/1059 [==============================] 

1059/1059 [==============================] - 2s 2ms/step - loss: 9.2697 - acc: 0.0918 - val_loss: 9.4451 - val_acc: 0.0802
Epoch 62/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2336 - acc: 0.0915 - val_loss: 9.4371 - val_acc: 0.0805
Epoch 63/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2653 - acc: 0.0891 - val_loss: 9.4290 - val_acc: 0.0813
Epoch 64/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2714 - acc: 0.0924 - val_loss: 9.4207 - val_acc: 0.0821
Epoch 65/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2646 - acc: 0.0924 - val_loss: 9.4123 - val_acc: 0.0831
Epoch 66/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2605 - acc: 0.0910 - val_loss: 9.4037 - val_acc: 0.0834
Epoch 67/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9.2377 - acc: 0.0924 - val_loss: 9.3950 - val_acc: 0.0839
Epoch 68/200
1059/1059 [==============================] - 2s 

1059/1059 [==============================] - 2s 2ms/step - loss: 8.5611 - acc: 0.1311 - val_loss: 8.6981 - val_acc: 0.1198
Epoch 122/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.5121 - acc: 0.1330 - val_loss: 8.6823 - val_acc: 0.1214
Epoch 123/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.4959 - acc: 0.1310 - val_loss: 8.6665 - val_acc: 0.1222
Epoch 124/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.5064 - acc: 0.1330 - val_loss: 8.6507 - val_acc: 0.1232
Epoch 125/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.4843 - acc: 0.1343 - val_loss: 8.6349 - val_acc: 0.1243
Epoch 126/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.4653 - acc: 0.1359 - val_loss: 8.6193 - val_acc: 0.1256
Epoch 127/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8.4584 - acc: 0.1409 - val_loss: 8.6036 - val_acc: 0.1275
Epoch 128/200
1059/1059 [==============================

In [16]:
baseline.evaluate(x_test_tfidf, y_test_tfidf, verbose=1)

295/295 [==============================] - 0s 1ms/step - loss: 8.1414 - acc: 0.1479


[8.141371726989746, 0.14791201055049896]

In [17]:
predict = baseline.predict(x_test_tfidf)

In [18]:
#predict_ = np.where(predict>0.55, 1, 0)
scoring(y_test_tfidf, predict)

,Threshold,F1_micro,Jaccard_micro
62,0.92,0.167133,0.091187


In [19]:
predict[1]

array([0.7831337 , 0.4360375 , 0.52132756, 0.73985654, 0.9072598 ,
       0.34264836, 0.29580933, 0.52294415, 0.63381034, 0.48198608,
       0.53523386, 0.39627838, 0.50225943, 0.84018433, 0.6914893 ,
       0.5729788 , 0.48741403, 0.3858711 , 0.65917736, 0.38613766,
       0.43865415, 0.8870549 , 0.9443282 , 0.88976693, 0.42277864,
       0.5158559 , 0.46581185, 0.46592662, 0.42864525, 0.4127088 ,
       0.35591227, 0.38705894, 0.3507833 , 0.39953735, 0.820135  ,
       0.41156927, 0.45887703, 0.6077236 , 0.4030287 , 0.3624907 ,
       0.64357275, 0.47348893, 0.40697816, 0.38519642, 0.4671661 ,
       0.508009  , 0.5488398 , 0.46940106, 0.3810812 , 0.55804515,
       0.4285453 , 0.31345358, 0.46241042, 0.4692177 , 0.41936842,
       0.53248256, 0.41279435, 0.3765208 , 0.48355323, 0.35148123,
       0.37141368, 0.43885556, 0.5189934 , 0.45058924, 0.41268662,
       0.57056963, 0.37420473, 0.45600903, 0.49669603, 0.41327703,
       0.3378682 , 0.33618075, 0.42518148, 0.38553876, 0.89479

# Neural Network "scratch" with Bert Tokenizer

## Tokenizer

In [20]:
# x_train, x_test split
x_train, x_test, y_train, y_test = train_test_split(x_unstemmed,
                                                    y_encoded,
                                                    test_size=0.2,
                                                    random_state=42)

In [21]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
# Ready output data for the model
max_length=250

x_train_toke = tokenizer(text=x_train.to_list(),
                         add_special_tokens=True,
                         max_length=max_length,
                         truncation=True,
                         padding=True, 
                         return_tensors='tf',
                         return_token_type_ids=False,
                         return_attention_mask=False,
                         verbose=True)

x_test_toke = tokenizer(text=x_test.to_list(),
                        add_special_tokens=True,
                        max_length=max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='tf',
                        return_token_type_ids=False,
                        return_attention_mask=False,
                        verbose=True)

## Model

In [23]:
# Define the model
xInput = Input(shape=(max_length))
x_ = Dense(700, activation='relu')(xInput)
x_ = Dense(700, activation='relu')(x_)
output = Dense(len(y_encoded[0]), activation='sigmoid')(x_)

model_toke = Model(inputs=xInput, outputs=output, name='Toke')

# Compile the model
model_toke.compile(loss=CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
                   optimizer=Adam(learning_rate=0.00000005),
                   metrics=[CategoricalAccuracy('accuracy')])

In [24]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                         mode='max',
                                         restore_best_weights=True)

In [25]:
model_toke.summary()

Model: "Toke"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 700)               175700    
_________________________________________________________________
dense_4 (Dense)              (None, 700)               490700    
_________________________________________________________________
dense_5 (Dense)              (None, 206)               144406    
Total params: 810,806
Trainable params: 810,806
Non-trainable params: 0
_________________________________________________________________


In [26]:
epochs = 200
batch_size = 64

In [27]:
# Load the model, if does not exist then train one
if globalStrategy == 'retrain' or globalStrategy == 'retrainToke':
    epochs = epochs
    batch_size=batch_size
    history = model_toke.fit(x_train_toke["input_ids"], y_train,
                             epochs=epochs,
                             validation_split=0.1,
                             callbacks=[callback],
                             verbose=1)

    model_toke.save('/home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/')
    model_toke = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/')

else:
    try:
        model_toke = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/')
    except OSError:
        epochs = epochs
        batch_size = batch_size
        history = model_toke.fit(x_train_toke["input_ids"], y_train,
                                 epochs=epochs,
                                 validation_split=0.1,
                                 callbacks=[callback],
                                 verbose=1)

        model_toke.save('/home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/')
        model_toke = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/')

Epoch 1/200
1059/1059 [==============================] - 2s 2ms/step - loss: 10815.0297 - accuracy: 0.0023 - val_loss: 10445.2637 - val_accuracy: 0.0027
Epoch 2/200
1059/1059 [==============================] - 2s 2ms/step - loss: 10349.4805 - accuracy: 0.0028 - val_loss: 10029.6533 - val_accuracy: 0.0027
Epoch 3/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9865.9587 - accuracy: 0.0033 - val_loss: 9662.8994 - val_accuracy: 0.0032
Epoch 4/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9557.3737 - accuracy: 0.0034 - val_loss: 9331.8984 - val_accuracy: 0.0032
Epoch 5/200
1059/1059 [==============================] - 2s 2ms/step - loss: 9129.3859 - accuracy: 0.0040 - val_loss: 9030.4053 - val_accuracy: 0.0035
Epoch 6/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8885.0665 - accuracy: 0.0037 - val_loss: 8752.3252 - val_accuracy: 0.0032
Epoch 7/200
1059/1059 [==============================] - 2s 2ms/step - loss: 8666.4084 - a

Epoch 55/200
1059/1059 [==============================] - 2s 2ms/step - loss: 5032.1105 - accuracy: 0.0410 - val_loss: 5111.3574 - val_accuracy: 0.0361
INFO:tensorflow:Assets written to: /home/mlmaster/Code/Ing_ml_P7/ModelTokenizer/assets


In [28]:
model_toke.evaluate(x_test_toke["input_ids"], y_test, verbose=1)

295/295 [==============================] - 0s 1ms/step - loss: 5282.8545 - accuracy: 0.0369


[5282.8544921875, 0.03687174618244171]

In [29]:
predict = model_toke.predict(x_test_toke["input_ids"])
# predict_ = np.where(predict > 0.5, 1, 0)
scoring(y_test, predict)

,Threshold,F1_micro,Jaccard_micro
68,0.98,0.024484,0.012394


In [30]:
predict[12]

array([0.0000000e+00, 0.0000000e+00, 8.2217391e-21, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.3500168e-12, 0.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 8.4029880e-27, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 9.9999857e-01, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e

In [31]:
print(x_train[15])
print(tokenizer.tokenize(x_train[15]))
print(x_train_toke["input_ids"][15])

read msg files read msg files need read outlook msg file net without com api outlook cos installed machines app run free 3rd party libraries want extract cc fields sent receive date fields would good also stored msg files
['read', 'ms', '##g', 'files', 'read', 'ms', '##g', 'files', 'need', 'read', 'outlook', 'ms', '##g', 'file', 'net', 'without', 'com', 'api', 'outlook', 'co', '##s', 'installed', 'machines', 'app', 'run', 'free', '3rd', 'party', 'libraries', 'want', 'extract', 'cc', 'fields', 'sent', 'receive', 'date', 'fields', 'would', 'good', 'also', 'stored', 'ms', '##g', 'files']
tf.Tensor(
[  101  8021  3787 10629  2028  2028 18750  8021  3787 10629  2028  2028
 18750  2699 20811 10629  9896 18750  3431  2191  2215  6140  3787 10629
  2028  2028  2878 10629  2393  2172 12315  4283  1038  2184  1038  2184
  1038  2184  2321  2531  2487  1038  6694  6694  6140  4130 13366 10629
  2707  2203  4130  4130  4130  2707  2707  2203  2709  4130 10629  2707
  2709 10425 13045 10629  2707 1

In [32]:
print(x_train[2])
print(tokenizer.tokenize(x_train[2]))
print(x_train_toke["input_ids"][2])

laravel use statement non name cache effect laravel use statement non name cache effect laravel 5 memcached try use cache routes php routes php line 3 use statement non name cache php cache
['lara', '##vel', 'use', 'statement', 'non', 'name', 'cache', 'effect', 'lara', '##vel', 'use', 'statement', 'non', 'name', 'cache', 'effect', 'lara', '##vel', '5', 'me', '##mc', '##ache', '##d', 'try', 'use', 'cache', 'routes', 'php', 'routes', 'php', 'line', '3', 'use', 'statement', 'non', 'name', 'cache', 'php', 'cache']
tf.Tensor(
[  101  6164 21025  2102  8833 21025  2102  4487  4246  6164 21025  2102
  8833 21025  2102  4487  4246  2667  4553 21025  2102  2393 21025  2102
  2028  2518  7188  2224 21025  2102  8833 21025  2102  4487  4246  3275
  2279  8087  2203  2773  2828 10954  2203  5494  2783 24234  3332  2330
  2178  2828  2279  3094  2215  2224   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     

# Bert Model with Bert Tokenizer

## Tokenizer

In [33]:
# Let's use the tokenizer we had before

## Bert

In [34]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config=config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [35]:
# Load bert layers
bert = transformer_model.layers[0]
bert

In [36]:
# Lock layers of bert so they don't get train again
bert.trainable = False

In [37]:
# Utiliser l'api functional
# Build the model
inputs = Input(shape=(max_length,), dtype='int32')
bert_model = bert(inputs)[1]
bert_model = Dropout(0.2)(bert_model, training=False)
bert_model = Dense(len(y_encoded[0]), activation='sigmoid', name='categ')(bert_model)

# Combine
myBert = Model(inputs=inputs, 
               outputs=bert_model, 
               name='BERT_MultiLabel_MultiClass')

In [38]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                         mode='max',
                                         restore_best_weights=True)

In [39]:
myBert.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
dropout_37 (Dropout)         (None, 768)               0         
_________________________________________________________________
categ (Dense)                (None, 206)               158414    
Total params: 109,640,654
Trainable params: 158,414
Non-trainable params: 109,482,240
_________________________________________________________________


In [40]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'categ': CategoricalCrossentropy(from_logits=True)}
metric = {'categ': CategoricalAccuracy('accuracy')}

# Compile the model
myBert.compile(
    optimizer=optimizer,
    loss=loss, 
    metrics=metric)

In [41]:
epochs = 200
batch_size=64

In [42]:
if globalStrategy == 'retrain' or globalStrategy == 'retrainBert':
    epochs = epochs
    batch_size=batch_size
    history = myBert.fit(x=x_train_toke["input_ids"],
                         y=y_train,validation_split=0.1,
                         batch_size=batch_size,
                         callbacks=[callback],
                         epochs=epochs)

    myBert.save('/home/mlmaster/Code/Ing_ml_P7/myBert/')
    myBert = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/myBert/')
else:
    try:
        myBert = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/myBert/')
    except OSError:
        epochs = epochs
        batch_size=batch_size
        history = myBert.fit(x=x_train_toke["input_ids"],
                             y=y_train,validation_split=0.1,
                             batch_size=batch_size,
                             callbacks=[callback],
                             epochs=epochs)

        myBert.save('/home/mlmaster/Code/Ing_ml_P7/myBert/')
        myBert = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/myBert/')

Epoch 1/200
530/530 [==============================] - 793s 1s/step - loss: 9.4803 - accuracy: 0.0314 - val_loss: 8.6683 - val_accuracy: 0.0454
Epoch 2/200
530/530 [==============================] - 792s 1s/step - loss: 8.4915 - accuracy: 0.0468 - val_loss: 8.5314 - val_accuracy: 0.0481
Epoch 3/200
530/530 [==============================] - 792s 1s/step - loss: 8.3208 - accuracy: 0.0485 - val_loss: 8.5022 - val_accuracy: 0.0494
Epoch 4/200
530/530 [==============================] - 793s 1s/step - loss: 8.3743 - accuracy: 0.0542 - val_loss: 8.4980 - val_accuracy: 0.0486
Epoch 5/200
530/530 [==============================] - 795s 1s/step - loss: 8.3246 - accuracy: 0.0539 - val_loss: 8.5019 - val_accuracy: 0.0494
Epoch 6/200
530/530 [==============================] - 795s 1s/step - loss: 8.3057 - accuracy: 0.0534 - val_loss: 8.5085 - val_accuracy: 0.0491
Epoch 7/200
530/530 [==============================] - 795s 1s/step - loss: 8.3596 - accuracy: 0.0563 - val_loss: 8.5159 - val_accuracy:

INFO:tensorflow:Assets written to: /home/mlmaster/Code/Ing_ml_P7/myBert/assets


INFO:tensorflow:Assets written to: /home/mlmaster/Code/Ing_ml_P7/myBert/assets


In [43]:
# Run evaluation
model_eval = myBert.evaluate(x=x_test_toke["input_ids"],
                             y=y_test)
model_eval

295/295 [==============================] - 184s 621ms/step - loss: 8.5197 - accuracy: 0.0600


[8.519692420959473, 0.060036126524209976]

In [44]:
predict = myBert.predict(x_test_toke["input_ids"])

In [45]:
# predict_ = np.where(predict > 0.95, 1, 0) # Threshold is very high but necessary as model outputs > 0.5 for all
scoring(y_test, predict)

,Threshold,F1_micro,Jaccard_micro
68,0.98,0.116102,0.061628


In [46]:
predict[0]

array([0.978031  , 0.65992993, 0.8706724 , 0.8981209 , 0.97752416,
       0.47003224, 0.47338086, 0.69648725, 0.8028426 , 0.7732695 ,
       0.87529725, 0.7327794 , 0.56030226, 0.97619015, 0.95441955,
       0.9061052 , 0.6354614 , 0.60939306, 0.9415106 , 0.6028231 ,
       0.469499  , 0.9836626 , 0.991642  , 0.9880163 , 0.75687283,
       0.838966  , 0.57190037, 0.63525856, 0.6838393 , 0.75669706,
       0.60401016, 0.5906911 , 0.5866819 , 0.59073126, 0.98210883,
       0.64273185, 0.61359364, 0.9288827 , 0.6943421 , 0.82769406,
       0.8117501 , 0.800745  , 0.6270091 , 0.5690289 , 0.7658088 ,
       0.73644376, 0.65136987, 0.8059165 , 0.6456533 , 0.7654775 ,
       0.702444  , 0.60459274, 0.7619956 , 0.5977242 , 0.49849996,
       0.81299084, 0.53426045, 0.6918267 , 0.83873296, 0.6689777 ,
       0.4455477 , 0.589231  , 0.79841715, 0.75317484, 0.5828179 ,
       0.8485511 , 0.58634484, 0.79908854, 0.86458164, 0.6054993 ,
       0.55012584, 0.6212171 , 0.55838066, 0.6405807 , 0.99234